In [2]:
import pyawr.mwoffice as mwo
import numpy as np
import matplotlib.pyplot as plt

from scipy.signal import savgol_filter
from matplotlib.animation import FuncAnimation 

'***PICK ONE***'
#Agg rendering embedded in a Jupyter widget. (inline) Requires ipympl:
# %matplotlib ipympl 
#Agg rendering to a Tk canvas (new window) Requires TkInter:
%matplotlib tk 

In [3]:
awrde = mwo.CMWOffice() #Create awrde object
awrde.Project.Simulator.Analyze() #Invoke circuit simulator analysis

In [4]:
def reset_freqs (l_bnd=4e9, u_bnd=8e9, steps=10000):
    '''Reset the MWO project frequencies.

    returns an array running from 
    lower_bound to upper_bound in steps steps.'''
    
    awrde.Project.Frequencies.Clear() # clear the frequencies specified for the project
    freq_arr = np.linspace(l_bnd, u_bnd, steps)
    awrde.Project.Frequencies.AddMultiple(freq_arr) # add the frequencies in the passed frequency array

    return freq_arr

In [5]:
def set_circ_params(circ_name='Inductor_Subcircuit', **kwargs): 
    '''Sets specified circuit parameters in circuit called circ_name.
    

    If you attempt to pass a parameter that doesn't exit in the circuit called circ_name, 
    MWO will throw an error.

    Returns a dictionary with the new values.
    '''

    passed_circ_param_vals = { # dictionary of subcircuit element parameter values (e.g. the value of the capacitor (element's) capacitance (parameter.))
        'CAP': awrde.Project.Schematics(circ_name).Elements(1).Parameters(2).ValueAsDouble,
        'IND': awrde.Project.Schematics(circ_name).Elements(2).Parameters(2).ValueAsDouble,
        'RES': awrde.Project.Schematics(circ_name).Elements(3).Parameters(2).ValueAsDouble
    }
    passed_circ_params = { # dictionary of subcircuit component 
        'CAP': awrde.Project.Schematics(circ_name).Elements(1).Parameters(2),
        'IND': awrde.Project.Schematics(circ_name).Elements(2).Parameters(2),
        'RES': awrde.Project.Schematics(circ_name).Elements(3).Parameters(2)
    }

    print("Setting:" + kwargs.__str__())

    new_circ_param_vals = {**passed_circ_param_vals, **kwargs} # in the case of duplicate keys, only the later key-value pair is preserved
    # print("DEBUG: new parameters:" + new_circ_param_vals.__str__())

    for i, value in enumerate(new_circ_param_vals.values()):
        list(passed_circ_params.values())[i].ValueAsDouble = value # in python3, dict.keys(), .values(), and .items() return dynamically changing view objects, but not the objects themselves. Hence, list().

        # print("DEBUG: " + list(passed_circ_param_vals)[i] + " set to " + str(value))
    
    return new_circ_param_vals

In [6]:
# ***********************************************
# *                Plotting S11                 *
# ***********************************************

def plot_graph(graph_name='Parallel', show_max=False, *args):
    graph = awrde.Project.Graphs(graph_name)
    meas = graph.Measurements[0]

    num_pts = meas.XPointCount
    xs = ys = dys = np.zeros(num_pts)

    def animate_smooth(n_frm): #Animate method for FuncAnimation
        xs = np.asarray(meas.XValues)
        ys = np.asarray(meas.YValues(1))
        ys_savgol = savgol_filter(ys, window_length=99, polyorder=3)
        dys_savgol = savgol_filter(ys, window_length=99, polyorder=3, deriv=1)
        abs_dys_savgol = np.abs(dys_savgol)

        axs_smooth[0].cla() # clear the axes
        axs_smooth[1].cla() #

        if show_max:
            max_ind = np.argmax(abs_dys_savgol)
            max_x = xs[max_ind]
            max_y = ys[max_ind]
            max_dy = dys_savgol[max_ind]
            axs_smooth[0].plot(max_x, max_y, 'ro')

            label = "({:.4g}, 1.0+{:.4g},); deriv.={:.4g})".format(max_x, 1.0-max_y, max_dy)
            axs_smooth[0].annotate(label, (max_x, max_y), textcoords='offset points', xytext=(5,2), ha='left', size=10)

        axs_smooth[0].plot(xs, ys_savgol, 'b') # applying a savgol filer
        axs_smooth[1].plot(xs, abs_dys_savgol, 'r') # applying a savgol filter, with differentiation
        # axs_smooth[0].plot(xs, ys, 'r')
        # axs_smooth[1].plot(xs, abs_dys, 'b')

    # TODO: Get zoom to work?
    fig_smooth, axs_smooth = plt.subplots(2, 1, sharex='all')
    
    #TODO: Figure out this blit thing
    ani = FuncAnimation(fig_smooth, animate_smooth, interval=500, blit=False) # create animation object. blit=True is for smoother animations, only changed data should be updated.
    fig_smooth.suptitle(graph_name)
    plt.show()
    return ani # Important! Without this, ani goes out of scope and the animation gets garbage collected!


# ***********************************************
# *           Plotting S11: Parallel            *
# ***********************************************

graph_name = 'Parallel'
ani1 = plot_graph(graph_name, show_max=True) 


# ***********************************************
# *            Plotting S11: Series             *
# ***********************************************

graph_name = 'Series'
ani2 = plot_graph(graph_name, show_max=True)

In [ ]:
# ***********************************************
# *    Specifiying the sweep of a parameter:    *
# ***********************************************

# ---------------USER-SETTABLE:-------------- #
# If a parameter is missing from the dictionary it will not be reset.
# If you want to keep something constant then comment it out.
# If you want to set it to a differnt constant then set lower_bound = upper_bound. 
# If you only want to change one variable at a time then... uhh... stay tuned.
#'COMPONENT' : (lower_bound, upper_bound)    
sample_circ_param_bounds = {
    'CAP': (2e-14,2e-12),
    # 'IND': (0,1),
    'RES': (1000000000,1000000000)
}

resolution = 10 # sets the number of parameter values to simulate (i.e. the resolution.)
# ------------------------------------------- #

sample_circ_param_arrays = dict.fromkeys(sample_circ_param_bounds)

keys = sample_circ_param_arrays.keys()
bnds = sample_circ_param_bounds.values()
arrs = list(np.linspace(*bnds_tuple, resolution) for bnds_tuple in bnds)

sample_circ_param_arrays.update(zip(keys, arrs))

# print("DEBUG: " + sample_circ_param_arrays.__str__())
 

# ***********************************************
# * Running the sumulation with set parameters: *
# ***********************************************

# FIXME: This loop actually changes all the elements simultaneously. Add some logic to do it iteratively...
for ind in range(resolution):
    indiv_vals = list(arr[ind] for arr in arrs)
    ele_val_dict = dict(zip(keys, indiv_vals))
    # FIXME: This is probably not great, first creating a dictionary then unpacking it immediately...
    set_circ_params(**ele_val_dict)

    # print('DEBUG: ' + ele_val_dict.__str__())
    

In [ ]:
def get_xs 

In [ ]:
def high_res_bounds():
    '''Focuses on the peak of S11 to get rid of noisy data'''
    

    graph = awrde.Project.Graphs(graph_name)
    meas = graph.Measurements[0]

    num_pts = meas.XPointCount
    xs = ys = dys = np.zeros(num_pts)